In [1]:
import json
import pandas as pd
import re

- Case no (here 746 F.3d 1045)
- Federal Circuit
- Name of Case 
- Case No (here no 2013-1239)
- Date
- Synapsis (Here the background paragraph)
- author (here the author linked in blue- O’Malley- this may be hard to do so not a big deal if hard to do
- Holdings: holdings para
- Result: here, affirmed impart vacated and remanded; it is usually the line after Holdings
-  the judge panel; later in the opinion they list the judges (note Before and then list the judges name)
- Also right above that they list the lawyers
- eliminate the cases that start with “In Re” from analysis

In [2]:
cases = json.load(open('/Users/liyannie/Desktop/Fall_2018_Courses/RA/amy_cases.json', 'r'))

In [3]:
len(cases.keys())

854

In [6]:
names = []
texts = []
for key, value in cases.items():
    key = key.replace('/Users/liyannie/Dropbox/Westlaw_cases/', '')
    key = key.replace('.pdf', '')
    key = re.sub("\d", '', key)
    key = key.replace('-', '')
    key = key.lstrip()
    names.append(key)
    texts.append(value)
dict1 = {'names': names, 'texts':texts} 
df = pd.DataFrame({'texts':texts}, index = names)

In [7]:
df.head()

,texts
Apple Inc v International Trade Com'n,"Apple Inc. v. International Trade Com'n, 725 F..."
Regeneron Pharmaceuticals Inc v Merus NV,"Regeneron Pharmaceuticals, Inc. v. Merus N.V.,..."
Joseph Phelps Vineyards LLC v Fairmont Holdings LLC,"Joseph Phelps Vineyards, LLC v. Fairmont Holdi..."
Commonwealth Scientific and Indus Research Organisation v Cisco Systems In,Commonwealth Scientific and Indus. Research Or...
Radio Systems Corp v Lalor,"Radio Systems Corp. v. Lalor, 709 F.3d 1124 (2..."


In [8]:
df.shape

(854, 1)

# eliminate cases start with "in re"

In [9]:
in_re = []
for name in df.index:
    if name.startswith('In re') == True:
        in_re.append('Y')
    else:
        in_re.append('N')
df['In_re'] = pd.Series(in_re).values
df.head(10)

,texts,In_re
Apple Inc v International Trade Com'n,"Apple Inc. v. International Trade Com'n, 725 F...",N
Regeneron Pharmaceuticals Inc v Merus NV,"Regeneron Pharmaceuticals, Inc. v. Merus N.V.,...",N
Joseph Phelps Vineyards LLC v Fairmont Holdings LLC,"Joseph Phelps Vineyards, LLC v. Fairmont Holdi...",N
Commonwealth Scientific and Indus Research Organisation v Cisco Systems In,Commonwealth Scientific and Indus. Research Or...,N
Radio Systems Corp v Lalor,"Radio Systems Corp. v. Lalor, 709 F.3d 1124 (2...",N
GE Lighting Solutions LLC v AgiLight Inc,"GE Lighting Solutions, LLC v. AgiLight, Inc., ...",N
United Construction Products Inc v Tile Tech Inc,"United Construction Products, Inc. v. Tile Tec...",N
Rosebud LMS Inc v Adobe Systems Inc,"Rosebud LMS Inc. v. Adobe Systems Inc., 812 F....",N
Norgren Inc v International Trade Com'n,"Norgren Inc. v. International Trade Com'n, 699...",N
UltimatePointer LLC v Nintendo Co Ltd,"UltimatePointer, L.L.C. v. Nintendo Co., Ltd.,...",N


In [10]:
df = df.loc[df.In_re != 'Y']

In [11]:
df.shape

(765, 2)

In [12]:
df.In_re.value_counts()

N    765
Name: In_re, dtype: int64

In [13]:
a = df.texts[0]
type(a)

str

In [14]:
df.head()

,texts,In_re
Apple Inc v International Trade Com'n,"Apple Inc. v. International Trade Com'n, 725 F...",N
Regeneron Pharmaceuticals Inc v Merus NV,"Regeneron Pharmaceuticals, Inc. v. Merus N.V.,...",N
Joseph Phelps Vineyards LLC v Fairmont Holdings LLC,"Joseph Phelps Vineyards, LLC v. Fairmont Holdi...",N
Commonwealth Scientific and Indus Research Organisation v Cisco Systems In,Commonwealth Scientific and Indus. Research Or...,N
Radio Systems Corp v Lalor,"Radio Systems Corp. v. Lalor, 709 F.3d 1124 (2...",N


# extract case name and 1st case number

In [15]:
def name_number(df):
    name_number = []
    for text in df.texts:
        name_num1 = re.findall(r'\A.*', text)
        name_number.append(name_num1)
    return name_number

# extract court

# extract 2nd case number

In [16]:
def case_num2(df):
    case_num2 = []
    for text in df.texts:
        case_num = re.match(r'((.*\n){4})Synopsis', text)[0]
        case_num2.append(case_num)
    return case_num2

In [185]:
case_num = re.match(r'((.*\n){4})Synopsis', a)[0]
case_num

TypeError: 'NoneType' object is not subscriptable

# extract date

In [17]:
def date(df):
    date = []
    for text in df.texts:
        date = re.match(r'((.*\n){2})Synopsis', text)[0]
        date.append(date)
    return date

# extract synopsis/background

In [18]:
def background(df):
    backgrounds = []
    for text in df.texts:
        background = re.findall(r'Synopsis(.*?)Holding', text, re.DOTALL)
        backgrounds.append(background)
    return backgrounds

In [263]:
len(background(df))

215

# extract author

In [19]:
def author(df):
    authors = []
    for text in df.texts:
        author = re.findall(r'Holdings: The Court of Appeals,(.*?) Judge', text, re.DOTALL)
        authors.append(author)
    return authors

In [313]:
len(author(df))

215

# extract holdings & results

In [20]:
def holding(df):
    holdings = []
    for text in df.texts:
        holding = re.findall(r'Holding(.*?)West Headnotes', text, re.DOTALL)
        holdings.append(holding)
    return holdings

In [273]:
len(holding(df))

215

# extract judge panel and lawyers

In [21]:
def lawyers_judge(df):
    lawyers_judges = []
    for text in df.texts:
        lawyers_judge = re.findall(r'Attorneys and Law Firms(.*?)Opinion', text, re.DOTALL)
        lawyers_judges.append(lawyers_judge)
    return lawyers_judges

In [275]:
len(lawyers_judge(df))

215

# Compile into final DataFrame

In [22]:
name_numbers = name_number(df)
#case_num2 = case_num2(df)
#date = date(df)
backgrounds = background(df)
authors = author(df)
holdings = holding(df)
lawyers_judges = lawyers_judge(df)

In [23]:
final_dict = {'name_numbers': name_numbers, #'case_num2': case_num2,
             #'date' : dates, 'background': backgrounds,
             'author': authors, 'holding': holdings,
             'lawyers_judge': lawyers_judges}
final_df = pd.DataFrame(final_dict, index = df.index)

In [26]:
final_df.head(20)

,name_numbers,author,holding,lawyers_judge
Apple Inc v International Trade Com'n,"[Apple Inc. v. International Trade Com'n, 725 ...","[ Moore, Circuit]","[s: The Court of Appeals, Moore, Circuit Judge...","[\n\n*1359 E. Joshua Rosenkranz, Orrick, Herr..."
Regeneron Pharmaceuticals Inc v Merus NV,"[Regeneron Pharmaceuticals, Inc. v. Merus N.V....",[],[],[]
Joseph Phelps Vineyards LLC v Fairmont Holdings LLC,"[Joseph Phelps Vineyards, LLC v. Fairmont Hold...",[],"[s, LLC, 857 F.3d 1323 (2017)\n122 U.S.P.Q.2d ...","[\n\nTHOMAS SCHNECK, Law Offices of Thomas Sch..."
Commonwealth Scientific and Indus Research Organisation v Cisco Systems In,[Commonwealth Scientific and Indus. Research O...,"[ Prost, Chief]","[s: The Court of Appeals, Prost, Chief Judge, ...","[\n\n*1297 Michael Ng, Kobre & Kim LLP, San F..."
Radio Systems Corp v Lalor,"[Radio Systems Corp. v. Lalor, 709 F.3d 1124 (...","[ Moore, Circuit]","[s: The Court of Appeals, Moore, Circuit Judge...","[\n\n*1125 R. Bradford Brittian, Merchant & G..."
GE Lighting Solutions LLC v AgiLight Inc,"[GE Lighting Solutions, LLC v. AgiLight, Inc.,...","[ Moore, Circuit]","[s: The Court of Appeals, Moore, Circuit Judge...","[\n\n*1307 Richard L. Rainey, Executive Couns..."
United Construction Products Inc v Tile Tech Inc,"[United Construction Products, Inc. v. Tile Te...","[ Wallach, Circuit]","[s: The Court of Appeals, Wallach, Circuit Jud...","[\n\nDAVID VON GUNTEN, von Gunten Law LL..."
Rosebud LMS Inc v Adobe Systems Inc,"[Rosebud LMS Inc. v. Adobe Systems Inc., 812 F...","[ Moore, Circuit]","[s: The Court of Appeals, Moore, Circuit Judge...","[\n\n*1072 Justin Chen, Ahmad, Zavitsanos..."
Norgren Inc v International Trade Com'n,"[Norgren Inc. v. International Trade Com'n, 69...","[ Linn, Circuit]","[s: The Court of Appeals, Linn, Circuit Judge,...","[\n\n*1319 Carl F. Manthei, The Ollila Law Gr..."
UltimatePointer LLC v Nintendo Co Ltd,"[UltimatePointer, L.L.C. v. Nintendo Co., Ltd....","[ Lourie, Circuit]","[s: The Court of Appeals, Lourie, Circuit Judg...","[\n\n*818 Charles John Rogers, Conley Ro..."


In [27]:
final_df.to_csv('/Users/liyannie/Desktop/Fall_2018_Courses/RA/extraction.csv')